In [1]:
import pandas as pd
import numpy as np
from gensim.models import Word2Vec
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("cleaned_data.csv")
df.drop(columns=df.columns.values[0:2], inplace=True)
df.dropna(inplace=True)
df.head()

/home/psachin/.local/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,cleaned text,joke
0,soooo glad circumcised corona covered foreskin...,1.0
1,know google platform recording bowel movements...,1.0
2,difference wife vacuum vacuum nt snore sex amp...,1.0
3,french man say attractive teletubby oo lala,1.0
4,yo mama yo momma fat went zoo hippos got jealous,1.0


In [3]:
df_jokes = df[df['joke']>0]
df_non_jokes = df[df['joke']<0]
df = pd.concat([df_jokes.sample(25000, random_state=42),
                df_non_jokes.sample(25000, random_state=42)])
df.head()

,cleaned text,joke
544835,walked bar saw fat bird dancing table went sai...,1.0
120452,band antivaxxers refuse listen cure,1.0
100104,call building got engaged premrs,1.0
501226,shit gross one best friend dad used tell us me...,1.0
752947,hear couple stole calendar got six months,1.0


In [4]:
X_train, X_test, y_train, y_test = train_test_split(df['cleaned text'], df['joke'], test_size=0.2, random_state=42)

In [5]:
X_train = [doc.split() for doc in X_train]
X_test = [doc.split() for doc in X_test]

In [6]:
w2v = Word2Vec(vector_size=200, window=5, min_count=4, workers=8)
k = w2v.build_vocab(X_train)

w2v.train(X_train, total_examples=len(X_train), epochs=32)

vectors = w2v.wv

In [7]:
new_X_train = pd.DataFrame()

for words in X_train:
    temp = pd.DataFrame()
    for word in words:
        if word in vectors:
            word_vec=vectors[word]
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
    
    doc_vector = temp.mean()
    new_X_train = new_X_train.append(doc_vector, ignore_index = True)

X_train = new_X_train
X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.283621,0.337758,0.374453,0.909987,0.153843,0.055045,0.210112,0.146602,-0.363172,0.027367,...,-0.069234,0.277983,0.326706,-0.383001,-0.433541,-0.042509,-0.356880,0.508626,0.121620,0.149328
1,-0.039872,0.107508,-0.168220,0.524480,0.244185,-0.053711,0.526280,-0.375397,-0.123167,0.120633,...,-0.423670,0.503230,-0.075885,-0.749662,-0.165246,0.227849,-0.270127,0.099939,-0.199834,0.118743
2,0.158202,-0.176163,-0.334855,-0.177998,0.128736,0.417267,0.138570,0.398593,0.287535,-0.016462,...,-0.306935,-0.158361,-0.508714,0.033835,-0.926046,-0.335094,-0.259315,0.192535,-0.206912,0.117974
3,-0.361247,-0.909919,0.034487,-0.147707,0.117236,-0.881148,0.785487,-0.206083,-1.097644,-0.019026,...,0.065694,0.814297,-0.425226,0.549037,-0.055518,0.075494,-0.196632,-0.036546,0.375700,0.159825
4,-0.090046,-0.025200,-0.797521,-0.000436,0.556524,-0.600837,0.605994,-0.709765,-0.055487,0.103671,...,0.025552,0.184136,-0.920483,-0.006100,0.581571,0.126250,-0.031224,-0.594102,-0.089011,-0.146019


In [8]:
new_X_test = pd.DataFrame()

for words in X_test:
    temp = pd.DataFrame()
    for word in words:
        if word in vectors:
            word_vec=vectors[word]
            temp = temp.append(pd.Series(word_vec), ignore_index = True)
    
    doc_vector = temp.mean()
    new_X_test = new_X_test.append(doc_vector, ignore_index = True)

X_test = new_X_test
X_test.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.191753,0.288973,0.247580,-0.042001,0.394053,-0.261415,0.386442,0.356878,0.249086,-0.726407,...,0.289306,0.081700,0.042400,-0.011659,0.031773,0.323535,-0.316973,-0.303285,-0.221888,0.412979
1,-0.451899,-0.791872,-0.442395,0.164301,0.377688,0.217620,0.113543,-0.059942,0.201949,-0.562213,...,-1.080461,0.729109,-0.833580,0.832179,-1.348479,-0.689872,-1.192101,0.897287,0.329111,0.265638
2,0.068307,0.135415,0.350003,0.367741,-0.055465,-0.326503,0.426363,0.267416,-0.401879,-0.381673,...,-0.066184,0.454472,-0.241243,0.073759,0.256563,0.299901,0.552605,0.226103,-0.094215,0.109464
3,-0.148776,0.412112,-0.217662,-0.117164,0.809389,-0.813973,0.794019,-0.578328,0.165938,-0.740103,...,0.598203,0.924582,-0.276797,0.395905,-0.609423,-0.129927,-0.244953,0.176270,0.100467,-0.779356
4,-0.667856,-0.230222,-0.194447,0.258269,-0.318361,-0.281466,0.165903,0.246196,0.410119,-0.124090,...,0.399716,0.325429,-0.424461,0.012939,0.451042,-0.651186,-0.213149,-0.086050,-0.680593,0.107228


In [10]:
#resetting indices as we have a new dataframe for x
y_train.index = [i for i in range(len(y_train))]
y_test.index = [i for i in range(len(y_test))]

In [12]:
X_train['label'] = y_train
X_test['label'] = y_test

X_train.head()

,0,1,2,3,4,5,6,7,8,9,...,191,192,193,194,195,196,197,198,199,label
0,0.283621,0.337758,0.374453,0.909987,0.153843,0.055045,0.210112,0.146602,-0.363172,0.027367,...,0.277983,0.326706,-0.383001,-0.433541,-0.042509,-0.356880,0.508626,0.121620,0.149328,-1.0
1,-0.039872,0.107508,-0.168220,0.524480,0.244185,-0.053711,0.526280,-0.375397,-0.123167,0.120633,...,0.503230,-0.075885,-0.749662,-0.165246,0.227849,-0.270127,0.099939,-0.199834,0.118743,-1.0
2,0.158202,-0.176163,-0.334855,-0.177998,0.128736,0.417267,0.138570,0.398593,0.287535,-0.016462,...,-0.158361,-0.508714,0.033835,-0.926046,-0.335094,-0.259315,0.192535,-0.206912,0.117974,-1.0
3,-0.361247,-0.909919,0.034487,-0.147707,0.117236,-0.881148,0.785487,-0.206083,-1.097644,-0.019026,...,0.814297,-0.425226,0.549037,-0.055518,0.075494,-0.196632,-0.036546,0.375700,0.159825,1.0
4,-0.090046,-0.025200,-0.797521,-0.000436,0.556524,-0.600837,0.605994,-0.709765,-0.055487,0.103671,...,0.184136,-0.920483,-0.006100,0.581571,0.126250,-0.031224,-0.594102,-0.089011,-0.146019,1.0


In [13]:
X_train.to_csv('Word2Vec train.csv')
X_test.to_csv('Word2Vec test.csv')